In [2]:
import pandas as pd
import numpy as np
import re
import string

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [5]:
test = pd.read_csv("test.csv",encoding = "ISO-8859-1")
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Cuento los links de cada tweet: (tengo que hacerlo antes de limpiarlos porque en la limpieza se pierden)

In [6]:
train['cantidad_links'] = train['text'].str.count('http')

In [7]:
test['cantidad_links'] = test['text'].str.count('http')

In [8]:
train.nlargest(30,'cantidad_links').head()

,id,keyword,location,text,target,cantidad_links
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw,0,4
7082,10143,upheaval,Attock,Ancient Mayan Tablet found via http://t.co/LmUMzkLtln http://t.co/yebxxAryBF http://t.co/SRRUqfffr6 http://t.co/CadzxAgMSI,0,4
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadrillion Time Bomb http://t.co/GhmmUj7GbE (vÂÃ_ http://t.co/u9LvvLzhYe) http://t.co/LyJ57pq3yX,0,3
1373,1980,bush%20fires,Trinidad and Tobago,Drought fuels bush fires in Jamaica - http://t.co/ZDtDqQbAHC http://t.co/PsQCNsVfgP - @JamaicaObserver @cnewslive RE https://t.co/6ZGef8J8Bm,1,3


## Signos de pregunta

In [9]:
test['cantidad_signos_pregunta'] = test['text'].str.count('\?')

In [10]:
train['cantidad_signos_pregunta'] = train['text'].str.count('\?')

## Limpieza de los textos

In [11]:
train_limpio = train.copy()

In [12]:
test_limpio = test.copy()

In [13]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()  #quito_mayusculas
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    return text

In [14]:
train_limpio['text'] = train_limpio['text'].apply(lambda x: clean_text(x))
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0


In [15]:
test_limpio['text'] = test_limpio['text'].apply(lambda x: clean_text(x))

## Analizar las repeticiones de un tweet

In [16]:
train_limpio['repeticiones'] = train_limpio.groupby('text')['id'].transform('count')
train_limpio.head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
0,1,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1,0,0,1
2,5,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1
3,6,NaN,NaN,people receive wildfires evacuation orders in california,1,0,0,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1


In [17]:
train_limpio.nlargest(20,'repeticiones').head()

,id,keyword,location,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
5876,8393,sandstorm,USA,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5877,8394,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5879,8397,sandstorm,NaN,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5882,8401,sandstorm,hkXfYMhEx,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22
5883,8402,sandstorm,US,watch this airport get swallowed up by a sandstorm in under a minute,1,1,0,22


In [18]:
test_limpio['repeticiones'] = test_limpio.groupby('text')['id'].transform('count')

# Remuevo informacion
## Remuevo el ID, y remuevo keyword y location, por ahora, despues me fijo que informacion les puedo sacar

In [19]:
train_limpio = train_limpio.drop(columns=['id'])
train_limpio = train_limpio.drop(columns=['keyword','location'])

In [20]:
test_limpio = test_limpio.drop(columns=['id'])
test_limpio = test_limpio.drop(columns=['keyword','location'])

## Remuevo los datos repetidos 
### No remuevo los repetivos del test obviamente

In [21]:
train_limpio.shape

(7613, 5)

In [22]:
train_limpio = train_limpio.drop_duplicates()
train_limpio.shape

(7134, 5)

In [23]:
train_limpio.head()

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1
1,forest fire near la ronge sask canada,1,0,0,1
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1
3,people receive wildfires evacuation orders in california,1,0,0,1
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1


# Longitud de los tweets

In [24]:
import math

def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [25]:
train_limpio['longitud_tweet'] = train_limpio['text'].apply(lambda x: roundup(len(x)))

In [26]:
train_limpio.head(2)

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,70
1,forest fire near la ronge sask canada,1,0,0,1,40


In [27]:
test_limpio['longitud_tweet'] = test_limpio['text'].apply(lambda x: roundup(len(x)))

## Contar la cantidad de veces seguidas que se repite una letra, si alguna se repite mas de 3 veces, es probable que haya sido intencional, lo cual le quita credibildad

In [28]:
def letras_repetidas(text):
    max_repeteated_letters = 0
    current_repeated_letters = 1
    for word in text:
        if(current_repeated_letters>max_repeteated_letters):
            max_repeteated_letters=current_repeated_letters
        current_repeated_letters=1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeteated_letters):
                    max_repeteated_letters=current_repeated_letters
                current_repeated_letters=1
    return (max_repeteated_letters)

In [29]:
def remover_palabras_con_letras_repetidas(text):
    max_repeated_letters = 0
    current_repeated_letters = 1
    new_text = []
    for word in text:
        current_repeated_letters=1
        max_repeated_letters = 1
        for i in range (len(word)-1):
            if(word[i]==word[i+1]):
                current_repeated_letters=current_repeated_letters+1
            else:
                if(current_repeated_letters>max_repeated_letters):
                    max_repeated_letters=current_repeated_letters
                current_repeated_letters=1
        if(current_repeated_letters<=2 and max_repeated_letters<=2):
            new_text.append(word)  #remuevo las palabras que tengan 3 o mas letras seguidas repetidas
    return ( ' '.join(new_text))

In [30]:
train_limpio['letras_seguidas'] = train_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

test_limpio['letras_seguidas'] = test_limpio['text'].apply(lambda x: letras_repetidas(x.split(' ')))

In [31]:
letras_seguidas_train = train_limpio['letras_seguidas']
letras_seguidas_test = test_limpio['letras_seguidas']

In [32]:
#Remuevo esas palabras de los textos
#train_limpio['text'] = train_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
#test_limpio['text'] = test_limpio['text'].apply(lambda x: remover_palabras_con_letras_repetidas(x.split(' ')))
#train_limpio.head(2)

### Parece dar buenos resultados pero obviamente no reemplaza un programa que me detecte cualquier error ortografico

# Correccion de typos

### Lo deje comentado por ser demasiado lento

In [33]:
#Prueba basica
#pruebita = train_limpio.iloc[1:150].copy()

def spell_correcter(text):
    word_list = text.split(' ')
    return [str(TextBlob(word).correct()) for word in word_list]

from textblob import TextBlob
#pruebita['text'] = pruebita['text'].apply(lambda x: spell_correcter(x))
#pruebita.text = pruebita.text.apply(lambda txt: ''.join(TextBlob(txt).correct()))

#pruebita['text_2'] = pruebita['text'].apply(lambda x: ' '.join(x))
#pruebita.head()

In [34]:
#Confirmé que funciona, lo aplico a todas

In [35]:
#train_limpio['text'] = train_limpio['text'].apply(lambda x: spell_correcter(x))

#train_limpio['text'] = train_limpio['text'].apply(lambda x: ' '.join(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: spell_correcter(x))

#test_limpio['text'] = test_limpio['text'].apply(lambda x: ' '.join(x))

In [36]:
#train_limpio.to_csv('train_con_textos_corregidos.csv', index=False)

#test_limpio.to_csv('test_con_textos_corregidos.csv', index=False)

In [37]:
#train_texto_corregido = pd.read_csv("train_con_textos_corregidos.csv",encoding = "ISO-8859-1")
#test_texto_corregido = pd.read_csv("test_con_textos_corregidos.csv",encoding = "ISO-8859-1")

In [38]:
#train_texto_corregido = train_texto_corregido.text.fillna(0)

In [39]:
#test_texto_corregido = test_texto_corregido.text.fillna(0)

In [40]:
#train_limpio['texto_corregido'] = train_texto_corregido

In [41]:
#test_limpio['texto_corregido'] = test_texto_corregido

In [42]:
#train_limpio[train_limpio['texto_corregido']!=train_limpio['text']].head()

# Termina corrigiendo typos que no deberia, asique no lo uso

# Analisis de sentimiento y de objetividad

In [43]:
b = TextBlob("Simple is better than complex")
b.sentiment

Sentiment(polarity=0.06666666666666667, subjectivity=0.41904761904761906)

In [44]:
#Prueba basica
pruebita = train_limpio.iloc[1:150].copy()
pruebita['analisis_sentimientos'] = pruebita['text'].apply(lambda x: (TextBlob(str(x)).sentiment))
pruebita.head()

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,analisis_sentimientos
1,forest fire near la ronge sask canada,1,0,0,1,40,1,"(0.1, 0.4)"
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1,130,2,"(-0.018750000000000003, 0.3875)"
3,people receive wildfires evacuation orders in california,1,0,0,1,60,1,"(0.0, 0.0)"
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1,90,2,"(0.0, 0.0)"
5,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1,0,0,1,110,1,"(-0.1125, 0.2375)"


In [45]:
train_limpio['sentimiento'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
train_limpio['objetividad'] = train_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [46]:
test_limpio['sentimiento'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[0])
test_limpio['objetividad'] = test_limpio['text'].apply(lambda x: (TextBlob(str(x)).sentiment)[1])

In [47]:
train_limpio.head()

,text,target,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,our deeds are the reason of this earthquake may allah forgive us all,1,0,0,1,70,2,0.00000,0.0000
1,forest fire near la ronge sask canada,1,0,0,1,40,1,0.10000,0.4000
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1,0,0,1,130,2,-0.01875,0.3875
3,people receive wildfires evacuation orders in california,1,0,0,1,60,1,0.00000,0.0000
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1,0,0,1,90,2,0.00000,0.0000


## Caso abarcativo: agrupo por palabras y me quedo con las mas n palabras mas repetidas.
## Por ahora me quedo con 100

In [48]:
valor_n = 200

In [49]:
import re 
from string import punctuation

palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()
palabras_por_separado.columns = ['word']
palabras_por_separado['word'] = palabras_por_separado['word'].str.replace(rf'[{punctuation}]', '')
palabras_por_separado.reset_index(inplace=True)
del palabras_por_separado['level_1']
palabras_mas_comunes = palabras_por_separado.groupby('word')['target'].count().rename('cantidad_apariciones').reset_index().nlargest(valor_n,'cantidad_apariciones')
palabras_mas_comunes.head()

,word,cantidad_apariciones
15299,the,3070
0,a,2036
15570,to,1853
7907,in,1816
10982,of,1674


In [50]:
palabras_mas_comunes

,word,cantidad_apariciones
15299,the,3070
0,a,2036
15570,to,1853
7907,in,1816
10982,of,1674
...,...,...
333,alabama,14
358,album,14
528,ancient,14
722,arent,14


In [51]:
for i in palabras_mas_comunes.word.values:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i))

In [52]:
test_limpio.head(2)

,text,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad,the,a,...,wasnt,worse,york,young,abc,alabama,album,ancient,arent,art
0,just happened a terrible car crash,0,0,1,40,2,-1.00,1.00,0,4,...,0,0,0,0,0,0,0,0,0,0
1,heard about earthquake is different cities stay safe everyone,0,0,1,70,2,0.25,0.55,0,6,...,0,0,0,0,0,0,0,0,0,1


## Similar al anterior: agrupo por 2-shingles y me quedo con los que mas veces aparecen 
### En shingles puede apreciarse tambien informacion valiosa

In [53]:
valor_n =50

In [54]:
palabras_por_separado = train_limpio.copy()
palabras_por_separado.set_index(['target'],inplace=True)
palabras_por_separado = palabras_por_separado['text'].str.split(expand=True).stack().to_frame()
palabras_por_separado.columns = ['word']
palabras_por_separado = palabras_por_separado.reset_index()
palabras_por_separado = palabras_por_separado[['word']]
palabras_por_separado.head()

,word
0,our
1,deeds
2,are
3,the
4,reason


In [55]:
palabras_shifteadas = train_limpio.copy()
palabras_shifteadas.set_index(['target'],inplace=True)
palabras_shifteadas = palabras_shifteadas['text'].str.split(expand=True).stack().to_frame()
palabras_shifteadas.columns = ['next_word']
palabras_shifteadas = palabras_shifteadas.reset_index()
palabras_shifteadas = palabras_shifteadas[['next_word']]
palabras_shifteadas = palabras_shifteadas.shift(-1)
palabras_shifteadas.head()

,next_word
0,deeds
1,are
2,the
3,reason
4,of


In [56]:
palabras_shifteadas.shape

(97878, 1)

In [57]:
dos_shingles = palabras_por_separado.merge(palabras_shifteadas, left_index=True, right_index=True)
dos_shingles.head()

,word,next_word
0,our,deeds
1,deeds,are
2,are,the
3,the,reason
4,reason,of


In [58]:
dos_shingles['shingle'] = dos_shingles['word']+' '+dos_shingles['next_word']
dos_shingles['valor'] = 1
dos_shingles.head()

,word,next_word,shingle,valor
0,our,deeds,our deeds,1
1,deeds,are,deeds are,1
2,are,the,are the,1
3,the,reason,the reason,1
4,reason,of,reason of,1


In [59]:
dos_shingles_agrupados = dos_shingles.groupby('shingle').valor.sum().rename('cantidad_apariciones').to_frame()
shingles_mas_comunes = dos_shingles_agrupados.nlargest(valor_n,'cantidad_apariciones')
shingles_mas_comunes.head()

,cantidad_apariciones
shingle,
in the,291
of the,234
on the,128
to the,120
in a,108


In [60]:
#for i in shingles_mas_comunes.index:
#    train_limpio[i]= train_limpio['text'].apply(lambda x: x.count(i))
#    test_limpio[i]= test_limpio['text'].apply(lambda x: x.count(i))

## Por lo visto los 2-shingles aportan muy poca informacion asique no los agrego

## Columna si contiene insultos, o la cantidad que tiene, es necesario agregarlos aparte y agruparlos por si algunos por separado no estan en las n palabras mas usadas

In [61]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
insults_list = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
mean_insult = [train[train['text'].str.contains(i)].target.mean() for i in insults_list]
times_insult_used = [len(train[train['text'].str.contains(i)]) for i in insults_list]

In [62]:
print(insults_list)
print(times_insult_used)
print(mean_insult)

['idiot', 'fuck', 'moron', 'scum', 'bitch', 'shit', 'bastard', 'cunt', 'ridicul', 'tramp']
[4, 67, 1, 1, 10, 57, 1, 4, 4, 1]
[0.5, 0.208955223880597, 0.0, 0.0, 0.0, 0.3157894736842105, 0.0, 0.0, 0.25, 1.0]


In [63]:
def insults(text):
    insults = ['idiot','fuck','moron','scum','bitch','shit','bastard','cunt','ridicul','tramp']
    amount = 0
    for word in text.split(' '):
        if(word in insults):
            amount = amount +1
    return(amount)

In [64]:
#train_limpio['insultos_por_tweet'] = train_limpio['text'].apply(lambda x: insults(x) )
test_limpio['insultos_por_tweet'] = test_limpio['text'].apply(lambda x: insults(x) )

In [65]:
#No deberia ser necesario, pero tengo que convertir esto a string, aunque ya lo sea para que ande
train_limpio['insultos_por_tweet'] = train_limpio['text'].apply(lambda x: insults(x) )

In [66]:
train_limpio[train_limpio['insultos_por_tweet']==1].target.mean()

0.25773195876288657

In [67]:
# Si bien algunos aparecen mucho menos que otros, en todos se cumple que estan por debajo del 0.5

In [68]:
# Remuevo estas palabras de los textos
# Al final no las remuevo, por que lo haria?
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in insults_list])

## Remover expresiones, es necesario agregarlos aparte y agruparlos por si algunos por separado no estan en las n palabras mas usadas

In [69]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
expressions = ['lol','omg','wtf','lmao','imo']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_expressions = [train[train['text'].str.contains(i)].target.mean() for i in expressions]
times_expressions_used = [len(train[train['text'].str.contains(i)]) for i in expressions]

In [70]:
print(expressions)
print(times_expressions_used)
print(mean_expressions)

['lol', 'omg', 'wtf', 'lmao', 'imo']
[54, 6, 8, 14, 15]
[0.12962962962962962, 0.3333333333333333, 0.375, 0.0, 0.3333333333333333]


In [71]:
def expressions_counter(text):
    expressions = ['lol','omg','wtf','lmao','imo']
    amount = 0
    for word in text:
        if(word in expressions):
            amount = amount +1
    return(amount)

In [72]:
train_limpio['expresiones_por_tweet'] = train_limpio['text'].apply(lambda x: expressions_counter(x))
test_limpio['expresiones_por_tweet'] = test_limpio['text'].apply(lambda x: expressions_counter(x) )

In [73]:
# Remuevo estas palabras de los textos
# Al final no las remuevo, por que lo haria?
#train_limpio['text']=train_limpio['text'].apply(lambda x: [item for item in x if item not in expressions])

## Cuento pronombres personales,esto se habia hecho en el TP1 y dio buenos resultados, es necesario agregarlos aparte y agruparlos por si algunos por separado no estan en las n palabras mas usadas

In [74]:
#Estoy usando el train original porque para este prueba simple uso los textos aun no separados por palabra
personal_pronouns = ['i','me','you','she','her','him','he','his','we','us','them','they','im']  #agregar mas,no importa que el train no tenga, si el test si,joya
mean_personal_pronouns = [train[train['text'].str.contains(i)].target.mean() for i in personal_pronouns]
times_personal_pronoun_used = [len(train[train['text'].str.contains(i)]) for i in personal_pronouns]

In [75]:
print(personal_pronouns)
print(mean_personal_pronouns)
print(times_personal_pronoun_used)

['i', 'me', 'you', 'she', 'her', 'him', 'he', 'his', 'we', 'us', 'them', 'they', 'im']
[0.4389975213439824, 0.43219954648526077, 0.2056921086675291, 0.4017857142857143, 0.42958579881656805, 0.7189189189189189, 0.4109311740890688, 0.37099494097807756, 0.4495515695067265, 0.4447797223898612, 0.3557692307692308, 0.3006134969325153, 0.5167630057803468]
[7262, 2205, 773, 224, 845, 185, 3952, 593, 892, 1657, 104, 163, 865]


In [76]:
for i in personal_pronouns:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.split(' ').count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.split(' ').count(i))

## Palabras mas comunes (esta parte no se si deberia quedar así basandome en las del TP1 o rehacer eso ahora que está mas limpio el train o hacerlo de otra forma)

In [77]:
palabras = ['hiroshima','bombing','kill','california','suicide','fire','forest','train','accident','disaster'\
                ,'new','crash','storm','police','nuclear','two','dead','died','years']
for i in palabras:
    train_limpio[i]= train_limpio['text'].apply(lambda x: x.split(' ').count(i))
    test_limpio[i]= test_limpio['text'].apply(lambda x: x.split(' ').count(i))

In [78]:
#muchas de estas palabras ya estaban agregadas por otros medios

## Guardo los csvs limpios para mas adelante

In [220]:
train_limpio.to_csv('train_limpio.csv', index=False)

In [221]:
test_limpio.to_csv('test_limpio.csv', index=False)

## Redes neuronales con tensorFlow (lo pongo aca xq no consegui que ande)

In [157]:
#import tensorflow as tf
#tf.__version__

In [158]:
#mnist = tf.keras.datasets.mnist
#(x_train, y_train) , (x_test, y_test) = mnist.load_data()

In [159]:
#import matplotlib.pyplot as plt
#plt.imshow(x_train[0])
#plt.show()

In [160]:
#normalizo la data para reder neuronales para que prediga mejor
#x_train = tf.keras.utils.normalize(x_train, axis=1)
#x_test = tf.keras.utils.normalize(x_test, axis=1)

In [161]:
# armo el modelo

## Vectorizo los textos (quiza la parte mas importante de todo el cleaning y apenas lo toque)

In [162]:
train_solo_texto = train_limpio[['id','text']].copy()

KeyError: "['id'] not in index"

In [ ]:
train_solo_texto['text'] = train_solo_texto['text'].apply(lambda x: ' '.join(x))

In [ ]:
train_solo_texto.head()

In [ ]:
train_solo_texto.groupby('text')['id'].transform('count')

In [ ]:
train_solo_texto.groupby('text').id.count().to_frame().reset_index().nlargest(40,'id')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#agregarle n-shingles despues
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(train_solo_texto.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = train_solo_texto.index
data_dtm

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokens]
#train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)

In [ ]:
train_solo_texto['stemmed'] = train_solo_texto['text'].apply(lambda x: porter.stem(word) for word in x)